In [12]:
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

In [14]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys

now = datetime.datetime.now

In [34]:
now = datetime.datetime.now

batch_size = 32
nb_classes = 4
nb_epoch = 50

# input image dimensions
img_rows, img_cols = 150, 150
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [35]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [63]:
K.image_data_format()

'channels_last'

In [46]:
def train_model(model, train, test, nb_classes):
    
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit_generator(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [64]:
# define two groups of layers: feature (convolutions) and classification (dense)
input_shape = (img_rows, img_cols,3)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,..., padding="valid")`
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


In [65]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True)

In [66]:
test_datagen = ImageDataGenerator()

In [67]:
!ls ../data

train  validation


In [68]:
train_generator = train_datagen.flow_from_directory(
        '../data/train',
        target_size=(150, 150),
        batch_size=1,
        class_mode='categorical')

Found 554 images belonging to 4 classes.


In [69]:
validation_generator = test_datagen.flow_from_directory(
        '../data/validation',
        target_size=(150, 150),
        batch_size=1,
        class_mode='categorical')

Found 188 images belonging to 4 classes.


In [70]:
model = Sequential(feature_layers + classification_layers)

In [71]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
t = now()

In [74]:
model.fit_generator(generator = train_generator,
                    steps_per_epoch = 24,
                    validation_data = validation_generator,
                    validation_steps = 6)

Epoch 1/1
24/24 [==============================] - 1s - loss: 10.7454 - acc: 0.3333 - val_loss: 13.4317 - val_acc: 0.1667


In [76]:
model.fit_generator(
        train_generator,
        steps_per_epoch=24,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=6)

Epoch 1/50
24/24 [==============================] - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/50
24/24 [==============================] - 1s - loss: 14.7749 - acc: 0.0833 - val_loss: 5.3727 - val_acc: 0.6667
Epoch 3/50
24/24 [==============================] - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/50
24/24 [==============================] - 1s - loss: 10.7454 - acc: 0.3333 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/50
24/24 [==============================] - 1s - loss: 10.7454 - acc: 0.3333 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/50
24/24 [==============================] - 1s - loss: 10.0738 - acc: 0.3750 - val_loss: 13.4317 - val_acc: 0.1667
Epoch 7/50
24/24 [==============================] - 1s - loss: 10.7454 - acc: 0.3333 - val_loss: 5.3727 - val_acc: 0.6667
Epoch 8/50
24/24 [==============================] - 1s - loss: 11.4170 - acc: 0.2917 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 9/50
2

In [77]:
train_generator

In [78]:
model

In [79]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_17 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
activation_18 (Activation)   (None, 146, 146, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 73, 73, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 170528)            0         
__________

In [80]:
from keras.applications import VGG16
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import os